# start Java Virtual Machine for zemberek

In [32]:
pip install jpype1

In [33]:
import jpype

In [34]:
import jpype
from jpype import getDefaultJVMPath,JClass
jar = r"drive/MyDrive/nlp/zemberek/zemberek-full.jar"
if not jpype.isJVMStarted():
    jpype.startJVM(getDefaultJVMPath(), classpath=[jar])

In [35]:
TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
TurkishSpellChecker = JClass('zemberek.normalization.TurkishSpellChecker')
TurkishSentenceNormalizer = JClass('zemberek.normalization.TurkishSentenceNormalizer')
Paths = JClass('java.nio.file.Paths')
lookupRoot = Paths.get("drive/MyDrive/nlp/normalization")
lmPath = Paths.get("drive/MyDrive/nlp/data/lm/lm.2gram.slm")
morphology = TurkishMorphology.createWithDefaults()
morph = TurkishMorphology.createWithDefaults()
spell = TurkishSpellChecker(morph)
#normalizer = TurkishSentenceNormalizer(morphology, lookupRoot, lmPath)

# stopwords, normalization, lemmatization, stemming 

In [ ]:
data['preProcessing'] = data['preProcessing'].str.replace('i̇','i')

In [38]:
import nltk
from nltk.corpus import stopwords
from string import punctuation, digits
import itertools
from pathlib import Path
import re
import os
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
import pandas as pd
from collections import Counter
from pathlib import Path
import numpy as np

In [ ]:
import nltk
nltk.download("all")

In [40]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_word_list.remove('ama')

In [41]:
WORDS = dict()
spell_folder = Path(r"drive/MyDrive/nlp/zemberek")

def words(text): return re.findall(r'\w+', text.lower())
with open(os.path.expanduser(Path(spell_folder/ "big2.txt")), "r", encoding = 'utf-8') as f:
    for line in f:
        splitted = line.split()
        WORDS[splitted[0]] = int(splitted[1])

In [42]:
def lower(text):
    text=re.sub("İ","i",text)
    text = text.lower()
    return text

def resubComma(texts):
    texts= re.sub(","," ",texts)
    return texts

def vanish_punc(text):
    regex = re.compile('[%s]' % re.escape(punctuation))
    text = regex.sub(' ', text)
    return text

def vanish_digits(text):
    text=text.strip()
    vanish_digits = str.maketrans('', '', digits)
    text=text.translate(vanish_digits)
    return text

def dup_vanish(s1):
     return (''.join(i for i, _ in itertools.groupby(s1)))

def reverse(s): 
    if len(s) == 0: 
        return s 
    else: 
        return reverse(s[1:]) + s[0] 

def checkOpennes(word):
    vowels=['a','e','i','ı','o','ö','u','ü']
    open_vowels=['e','i','ü','ö']
    close_vowels=['a','ı','o','u']
    for i in range(len(word)):
        if reverse(word)[i] in vowels:
            if reverse(word)[i] in open_vowels:
                return True
            else:
                return False
        else:
            continue
def PresentCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if reverse(word)[i] in ei:
                return 'ei'
            elif reverse(word)[i] in üö:
                return 'üö'
            elif reverse(word)[i] in aı:
                return 'aı'
            elif reverse(word)[i] in uo:
                return 'uo'
            else:
                continue
    
def StartCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if (word)[i] in ei:
                return 'ei'
            elif (word)[i] in üö:
                return 'üö'
            elif (word)[i] in aı:
                return 'aı'
            elif (word)[i] in uo:
                return 'uo'
            else:
                continue

def correction(word):
      return max(candi(word), key=P)

In [43]:
def replaceall(s, n,a):
    occurence = s.count(n)
    alt = []
    temp = s
    for i in range(occurence):
        temp2 = temp
        for j in range(i,occurence):
            temp2 = temp2.replace(n,a,1)
            alt.append(temp2)
        temp = temp.replace(n,"!",1)
    for i in range(len(alt)):
        alt[i] = alt[i].replace("!",n)

    return alt

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    if  word in WORDS.keys():
        number = WORDS[word]
    else:
        number = 1
    if number == 0:
        number = 1
    return number / N

def correction(word):
      return max(candi(word), key=P)

def candi(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcçdefgğhıijklmnoöprsştuüvyzw'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]

    sp     = replaceall(word,'ı','i')
    sp2     = replaceall(word,'u','ü')
    sp3    = replaceall(word,'o','ö')
    sp4     = replaceall(word,'g','ğ')
    sp5     = replaceall(word,'c','ç')
    sp6     = replaceall(word,'s','ş')
    sp7     = replaceall(word,'i','ı')
    sp8     = replaceall(word,'ö','o')
    sp9     = replaceall(word,'ş','s')
    sp10     = replaceall(word,'ğ','g')
    sp11     = replaceall(word,'ç','c')
    sp12     = replaceall(word,'ü','u')
    specials=[]
    specials.extend(sp)
    specials.extend(sp2)
    specials.extend(sp3)
    specials.extend(sp4)
    specials.extend(sp5)
    specials.extend(sp6)
    specials.extend(sp7)
    specials.extend(sp8)
    specials.extend(sp9)
    specials.extend(sp10)
    specials.extend(sp11)
    specials.extend(sp12)
    return set(deletes+transposes+replaces+inserts+specials)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def print_diff(word, s):
    if not word == s:
        print(word + " --> " + s)
counter = 0

In [44]:
def lemmatizer(word,texts):
        wordList=[]
        wordList = re.sub("[^\w]", " ",  texts).split()
        if '�' in word:
            return 'question'
        pos=wordList.index(word)
        sakin=''
        word=correction(word)
        ## word=yaşında
        if len(wordList)-pos>3 and pos>2:
            for i, kelime in enumerate(wordList[pos-3:pos+4]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos>5:
            for i, kelime in enumerate(wordList[pos:pos+5]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos<=5:
            for i, kelime in enumerate(wordList[pos:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)-pos<1 and pos>3:
            for i, kelime in enumerate(wordList[pos-3:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)<3:
            for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        else:
             for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        results = morphology.analyze(word)
        lemma=[]
        form=[]
        l=[]
        m=[]
        for i, result in enumerate(results):
            form.append(str(result.formatLong()))
            lemma.append(result.getLemmas()[0])
        if len(lemma)>1:
                analysis = morphology.analyzeSentence(sakin)
                results = morphology.disambiguate(sakin, analysis).bestAnalysis()
                
                for i, result in enumerate(results):
                        l.append(result.getLemmas()[0])
                        m.append(result.formatLong())
                for i in range(len(m)):
                    for j in range(len(form)):
                        if m[i]==form[j]:
                            lema=lemma[j]
                            if lema=='değil':
                                return 'değil'
                            if 'Neg' in form[j] or 'WithoutHavingDoneSo' in form[j] or 'Unable' in form[j]:
                                if checkOpennes(word):
                                    return lema+'me'
                                else:
                                    return lema+'ma'
                            if 'Without' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'siz'
                                elif PresentCheck(word)=='aı':
                                    return lema+'sız'
                                elif PresentCheck(word)=='uo':
                                    return lema+'suz'
                                else:
                                    return lema+'süz'
                            if 'With' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'li'
                                elif PresentCheck(word)=='aı':
                                    return lema+'lı'
                                elif PresentCheck(word)=='uo':
                                    return lema+'lu'
                                else:
                                    return lema+'lü'
                            else:
                                return lema
                    else:
                        continue
        elif len(lemma)==1:
            if lemma[0]=='değil':
                return lemma[0]
            if 'Neg' in form[0] or 'WithoutHavingDoneSo' in form[0] or 'Unable' in form[0]:
                 if checkOpennes(word):
                    return lemma[0]+'me'
                 else:
                    return lemma[0]+'ma'
            elif 'Without' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'siz'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'sız'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'suz'
                else:
                    return lemma[0]+'süz'
            elif 'With' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'li'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'lı'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'lu'
                else:
                    return lemma[0]+'lü'
            else:
                return lemma[0]
        else:
            return word

In [45]:
def clean_messages(texts):
    texts=lower(texts)
    texts=resubComma(texts)
    tokens=WPT.tokenize(texts)
    text = [token for token in tokens if token not in stop_word_list]

    for i, word in enumerate(text):
        text[i]=dup_vanish(vanish_digits(vanish_punc((word))))

    text = list(filter(None,text))
    metin = ' '.join(text)

    for i, word in enumerate(text):
         if(word==" "):
           continue
         text[i] = lemmatizer(word,metin)

    return ' '.join(''.join(elems) for elems in text)


# Uygulama

In [136]:
## deneme
clean_messages("dişarida hava cok guzel")

'dışarı hava çok güzel'

In [ ]:
%%time
data["clean_messages"] = data["preProcessing"].map(clean_messages)